# Fairify Artifact
Artifact of the paper "Fairify: Fairness Verification of Neural Networks" from ICSE 2023.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [1]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [2]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 28 compute_cascadelake_r nodes ready to use


### Reserve node

In [3]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=8))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start...


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [4]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes for baremetal, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,597ac87a-79dd-4048-9321-dac4fc656790
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.194 (v4) Type: fixed MAC: bc:97:e1:78:f1:a0
Network Name,sharednet1
Created At,2025-06-19T10:06:17Z
Keypair,trovi-28d289c
Reservation Id,4c7d151d-3e18-409d-a646-0fd6f183072c
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [5]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.239 port 22.


Connection successful


# Setup nvidia drivers and cuda

Download drivers

In [ ]:
my_server.execute("""
sudo apt-get --purge remove -y '*cublas*' 'cuda*' 'nvidia-*' && \
sudo apt-get autoremove -y && \
sudo apt-get autoclean && \
sudo rm -rf /usr/local/cuda* && \
sudo apt update && \
sudo apt install -y nvidia-driver-470 && \
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb && \
sudo dpkg -i cuda-keyring_1.0-1_all.deb && \
sudo apt update && \
sudo apt install -y cuda-toolkit-11-2
""")

Restart nvidia services

In [ ]:
my_server.execute("sudo modprobe -r nouveau")
my_server.execute("sudo rmmod nouveau 2>/dev/null || true")
my_server.execute("sudo update-initramfs -u")
my_server.execute("sudo modprobe nvidia")

Verifiy services

In [ ]:
my_server.execute("nvidia-smi")
my_server.execute("""
if [ -L /usr/local/bin/nvcc ]; then sudo rm /usr/local/bin/nvcc; fi && \
sudo ln -s /usr/local/cuda-11.2/bin/nvcc /usr/local/bin/nvcc
""")
my_server.execute("nvcc --version")

## Run Fairify

Now, we can finally run Fairify. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [9]:
my_server.execute("rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git")

Cloning into 'Fairify'...


<Result cmd='rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git' exited=0>

In [10]:
my_server.execute("chmod +x Fairify/reproduce-experiment.sh")
my_server.execute("bash Fairify/reproduce-experiment.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Running `shell init`, which:
 - modifies RC file: "/home/cc/.bashrc"
 - generates config for root prefix: "/home/cc/miniconda"
 - sets mamba executable to: "/home/cc/miniconda/bin/mamba"
The following has been added in your "/home/cc/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba shell init' !!
export MAMBA_EXE='/home/cc/miniconda/bin/mamba';
export MAMBA_ROOT_PREFIX='/home/cc/miniconda';
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --root-prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    alias mamba="$MAMBA_EXE"  # Fallback on help from mamba activate
fi
unset __mamba_setup
# <<< mamba initialize <<<



warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/




Transaction

  Prefix: /home/cc/miniconda/envs/fairify

  Updating specs:

   - python=3.9


  Package               Version  Build           Channel         Size
───────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  main            pkgs/main     Cached
  + _openmp_mutex           5.1  1_gnu           pkgs/main     Cached
  + bzip2                 1.0.8  h5eee18b_6      pkgs/main     Cached
  + ca-certificates   2025.2.25  h06a4308_0      pkgs/main     Cached
  + expat                 2.7.1  h6a678d5_0      pkgs/main     Cached
  + ld_impl_linux-64       2.40  h12ee557_0      pkgs/main     Cached
  + libffi                3.4.4  h6a678d5_1      pkgs/main     Cached
  + libgcc-ng            11.2.0  h1234567_1      pkgs/main     Cached
  + libgomp              11.2.0  h1234567_1      pkgs/main     Cached
  + libstdcxx-ng         11.2.0  h1234567_1      

Reading package lists...
Building dependency tree...
Reading state information...
csvtool is already the newest version (2.4-1build3).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


...


Processing Models:  92%|█████████▏| 12/13 [30:44<02:33, 153.75s/it]

0       0
1       0
2       1
3       0
4       0
       ..
6779    0
6780    0
6781    0
6782    0
6783    1
Length: 6784, dtype: int64
True: 1659 | False: 5125
y_pred
0       False
1        True
2        True
3       False
4        True
        ...  
6779     True
6780    False
6781    False
6782     True
6783     True
Length: 6784, dtype: bool
True: 3809 | False: 2975
prot_attr
0       0.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
6779    1.0
6780    0.0
6781    0.0
6782    0.0
6783    1.0
Length: 6784, dtype: float64
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  5.992
['50', '1', '1', '11', '0', '0', '5', '3', '0', '10', '0', '32', '40']
['50', '1', '1', '11', '0', '0', '5', '3', '1', '10', '0', '32', '40']
[-0.01238179] [1.328438]
[-0.0123837] [1.328438]
pred1:  0.4969045915288853
pred2:  0.7905821515119286
class_1:  False
class_2:  True
pred1_orig:  0.49690411471000523
pred2_orig:  0.7905821515119286
class_1_orig:  False
c

Processing Models: 100%|██████████| 13/13 [30:44<00:00, 141.92s/it]


lass_2_orig:  True
******************
7 column
0       0.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
6779    1.0
6780    0.0
6781    0.0
6782    0.0
6783    1.0
Name: 8, Length: 6784, dtype: float64
y_true
0       0
1       0
2       1
3       0
4       0
       ..
6779    0
6780    0
6781    0
6782    0
6783    1
Length: 6784, dtype: int64
True: 1659 | False: 5125
y_pred
0       False
1        True
2        True
3       False
4        True
        ...  
6779     True
6780    False
6781    False
6782     True
6783     True
Length: 6784, dtype: bool
True: 3809 | False: 2975
prot_attr
0       0.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
6779    1.0
6780    0.0
6781    0.0
6782    0.0
6783    1.0
Length: 6784, dtype: float64
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  1.415
['61', '1', '15', '14', '3', '12', '0', '4', '1', '17', '19', '81', '21']
['61', '1', '15', '14', '3', '12', '0', '4', '0', '17', '19', '81

<Result cmd='bash Fairify/reproduce-experiment.sh' exited=0>

### Run dependencies and first experiment

Results for first experiment

In [ ]:
from chi import storage

bucket_name = "bare_metal_experiment_pattern_data"
b = storage.ObjectBucket(bucket_name)

print(f"Listing objects in bucket '{bucket_name}'...")
for obj in b.list_objects():
    print(f"Downloading {obj.name}")
    obj.download(obj.name) 

In [ ]:
my_server.execute("csvtool readable Fairify/src/AC/res/counterexample-adult-new.csv | less -S")

In [ ]:
my_server.execute("csvtool readable Fairify/src/AC/res/AC-14.csv | less -S")

In [11]:
my_server.execute("csvtool readable Fairify/src/AC/res/synthetic-adult-predicted-AC-metrics.csv | less -S")

Partition ID Original Accuracy  Original F1 Score  Pruned Accuracy Pruned F1 DI                 SPD                 EOD                  AOD                  ERD                  CNT          TI
1            0.6017099056603774 0.5058522311631309 1.0             1.0       0.6085714285714285 -0.2523956237424547 -0.14450961225154768 -0.17289090871216137 -0.08342555331991952 [0.87626769] 0.10114661109547711
2            0.6017099056603774 0.5058522311631309 1.0             1.0       0.6085714285714285 -0.2523956237424547 -0.14450961225154768 -0.17289090871216137 -0.08342555331991952 [0.87626769] 0.10114661109547711
3            0.6017099056603774 0.5058522311631309 1.0             1.0       0.6085714285714285 -0.2523956237424547 -0.14450961225154768 -0.17289090871216137 -0.08342555331991952 [0.87626769] 0.10114661109547711
4            0.6017099056603774 0.5058522311631309 1.0             1.0       0.6085714285714285 -0.2523956237424547 -0.14450961225154768 -0.17289090871216137 -0.08342555

<Result cmd='csvtool readable Fairify/src/AC/res/synthetic-adult-predicted-AC-metrics.csv | less -S' exited=0>